# Enlargement of a data set for object detection

This notebook can be used to enlarge a data set for object detection consisting of .jpg images and .xml files. The [Albumentations](https://albumentations.ai/docs/) library, which is a library for image augmentation, is used to modify the images.
The following sources were used to create the notebook:


*   https://chatgpt.com/
*   https://www.kaggle.com/code/ankursingh12/data-augmentation-for-object-detection/notebook
*   https://albumentations.ai/docs/


Upload your data set in a zipped folder called "images.zip". It should contain the images (.jpg) and the .xml files containing the annotations for the images. The annotations should be done in the Pascal VOC format.



In [ ]:
!unzip images.zip

In [ ]:
import os
images_zip = '/content/images.zip'
os.remove(images_zip)
print(f"Folder '{images_zip}' and its contents have been deleted.")

In [ ]:
!wget https://raw.githubusercontent.com/Joanna12105/smb-safety_system/main/tools/xml_to_csv.py
!wget https://raw.githubusercontent.com/Joanna12105/smb-safety_system/main/tools/csv_to_xml.py

In [ ]:
!python /content/xml_to_csv.py

In [ ]:
# ===========================================================================================================
# ================================================= IMPORTS =================================================
# ===========================================================================================================
import cv2
import ast
import pandas as pd
import numpy as np
import albumentations as A

train_folder = '/content/images/'


# ===========================================================================================================
# ================================================ FUNCTIONS ================================================
# ===========================================================================================================


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Function name:        load_data
# Function Parameter:   None
# Description:          Function to load data from the previously
#                        created .csv and to extract the bounding box
#                        information
# Return:               train_df > csv data loaded into a data frame
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def load_data():
    train_df = pd.read_csv('/content/org_imgs_bb.csv')
    bboxes = np.stack(train_df['Bounding Box'].apply(lambda x: ast.literal_eval(x)))
    for i, col in enumerate(['x_min', 'y_min', 'x_max', 'y_max']):
        train_df[col] = bboxes[:, i]
    return train_df


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Function name:        read_img
# Function Parameter:   img_id > unique image name, taken from the row
#                        "Filename" in the data frame
# Description:          Function to read an image from the specified
#                        folder (train_folder) based on the provided
#                        "img_id"
# Return:               img > image data
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def read_img(img_id):
    img_path = f"{train_folder}/{img_id}.jpg"
    print(f"Image read: {img_path}")
    img = cv2.imread(str(img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Function name:        read_bboxes
# Function Parameter:   img_id > unique image name, taken from the row
#                        "Filename" in the data frame
#                       df > data frame
#                       img_shape > height and width of the image
# Description:          Function to get the bounding box information
#                        and to normalize the bounding box data
#                        for a certain image, defined by the image id
# Return:               bboxes > normalized bounding box data
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def read_bboxes(img_id, df, img_shape):
    bboxes = df.loc[df.Filename == img_id, ['x_min', 'y_min', 'x_max', 'y_max']].values.astype(float)
    height, width = img_shape[:2]
    bboxes[:, 0] /= width
    bboxes[:, 1] /= height
    bboxes[:, 2] /= width
    bboxes[:, 3] /= height
    return bboxes


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Function name:        save_image
# Function Parameter:   image > contains the image information
#                       save_path > path under which the images are
#                        stored at
# Description:          Function to save an image to the specified
#                        path
# Return:               None
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def save_image(image, save_path):
    img_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.imwrite(save_path, img_bgr)
    print(f"Image saved: {save_path}")


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Function name:        organize_image_saving_and_annotation_data
# Function Parameter:   image > contains the image information
#                       save_path > path under which the images are
#                        stored at
#                       img_id > unique image name, taken from the row
#                        "Filename" in the data frame
#                       augmentation_name > stores the name of the
#                        applied augmentation
#                       bboxes > normalized bounding box data
#                       original_df > the data frame that was the
#                        "latest" state before the current image
#                        was processed
#                       bbox_indices > stores the list of indices in
#                        the original_df DataFrame corresponding to
#                        the bounding boxes of the current image
# Description:          Function to organize the saving of the newly
#                        created images and to update the dataframe
#                        with the new bounding box annotations
# Return:               original_df > the originally loaded data frame
#                        gets modified by concatenating it with the
#                        newly created data frame (new_df), which
#                        contains the new annotations (for each image
#                        the information gets appended)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def organize_image_saving_and_annotation_data(image, img_id, augmentation_name, bboxes, original_df, bbox_indices):
    height, width = image.shape[:2]
    new_rows = []
    for i, bbox in enumerate(bboxes):
        x_min = int(bbox[0] * width)
        y_min = int(bbox[1] * height)
        x_max = int(bbox[2] * width)
        y_max = int(bbox[3] * height)

        save_path = str(f"{train_folder}/{img_id}_{augmentation_name}.jpg")
        save_image(image, save_path)

        original_index = bbox_indices[i]
        new_entry = {
            'Folder': original_df.loc[original_index, 'Folder'],
            'Filename': f"{img_id}_{augmentation_name}.jpg",
            'Path': save_path,
            'Source': original_df.loc[original_index, 'Source'],
            'Width': original_df.loc[original_index, 'Width'],
            'Height': original_df.loc[original_index, 'Height'],
            'Depth': original_df.loc[original_index, 'Depth'],
            'Segmented': original_df.loc[original_index, 'Segmented'],
            'Object Name': original_df.loc[original_index, 'Object Name'],
            'Object Pose': original_df.loc[original_index, 'Object Pose'],
            'Object Truncated': original_df.loc[original_index, 'Object Truncated'],
            'Object Difficult': original_df.loc[original_index, 'Object Difficult'],
            'Bounding Box': [x_min, y_min, x_max, y_max]
        }

        new_rows.append(new_entry)

    new_df = pd.DataFrame(new_rows)
    original_df = pd.concat([original_df, new_df], ignore_index=True)

    return original_df


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Function name:        organize_image_augmentation_and_saving
# Function Parameter:   image > contains the image information
#                       save_path > path under which the images are
#                        stored at
#                       img_id > unique image name, taken from the row
#                        "Filename" in the data frame
#                       bboxes > normalized bounding box data
#                       augmentation_list > list that contains the
#                        different augmentations that should be done
#                        for every image
#                       original_df > stores the latest data frame
# Description:          Function to apply the augmentations
#                        defined in the augmentation_list to the
#                        images, save each augmented image, and
#                        update the data frame ("original_df") with
#                        the new annotation data.
# Return:               original_df > stores the latest data frame
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def organize_image_augmentation_and_saving(image, img_id, bboxes, augmentation_list, original_df):
    bbox_indices = original_df.loc[original_df.Filename == f"{img_id}.jpg"].index.tolist()
    for aug_name, aug_func in augmentation_list:
        transformed_img, transformed_bboxes = apply_augmentation(image, bboxes, aug_func)
        original_df = organize_image_saving_and_annotation_data(transformed_img, img_id, aug_name, transformed_bboxes,
                                                                original_df, bbox_indices)

    return original_df


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Function name:        apply_augmentation
# Function Parameter:   image > contains the image information
#                       save_path > path under which the images are
#                        stored at
#                       bboxes > normalized bounding box data
#                       augmentation > stores the augmentation data
#                       original_df > stores the latest data frame
# Description:          Function to apply a given augmentation to an
#                        image and its bounding boxes.
# Return:               transformed_image > stores the augmented image
#                       transformed_bboxes > stores the data for the
#                        augmented bounding boxes for that image
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def apply_augmentation(image, bboxes, augmentation):
    transformed = augmentation(image=image, bboxes=bboxes, labels=np.ones(len(bboxes)))
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']
    return transformed_image, transformed_bboxes


# ===========================================================================================================
# ================================================= "MAIN" ==================================================
# ===========================================================================================================
df = load_data()

bbox_params = {'format': 'pascal_voc', 'label_fields': ['labels']}

albumentation_list = [
    ('RandomFog', A.Compose([A.RandomFog(p=1)], bbox_params=bbox_params)),
    ('RandomBrightness', A.Compose([A.RandomBrightness(p=1)], bbox_params=bbox_params)),
    ('RGBShift', A.Compose([A.RGBShift(p=1)], bbox_params=bbox_params)),
    ('RandomSnow', A.Compose([A.RandomSnow(p=1)], bbox_params=bbox_params)),
    ('RandomContrast', A.Compose([A.RandomContrast(limit=0.5, p=1)], bbox_params=bbox_params))
]

for img_id in df['Filename'].str[:-4].unique():
    chosen_img = read_img(img_id)
    bboxes = read_bboxes(f"{img_id}.jpg", df, chosen_img.shape)
    df = organize_image_augmentation_and_saving(chosen_img, img_id, bboxes, albumentation_list, df)

df.to_csv('/content/org_imgs_bb.csv', index=False)


In [ ]:
!python /content/csv_to_xml.py

In [ ]:
%cd /content/
!zip -r images.zip images

In [ ]:
%cd ..
%cd /home/
!zip -r dataset_improvement.zip /content

In [ ]:
from google.colab import files

files.download('/home/dataset_improvement.zip')